In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import functions as fn
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
import os
import math
from pyspark.sql.window import Window
import time
import datetime

os.environ["JAVA_HOME"] = "/workdir/data/JAVA8/jdk1.8.0_202"

# SparkContext.setSystemProperty("spark.driver.maxResultSize","1g")
spark = SparkSession.builder \
    .config("spark.excutor.memory", "300g") \
    .config("spark.driver.memory", "300g") \
    .config("spark.excutor.cores", 40) \
    .appName("Read Parquet") \
    .getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
year = 2023
month = 3
day = 1

now_date = year * 10000 + month * 100 + day
nex_date = year * 10000 + month * 100 + day + 1

date_time = datetime.datetime(year, month, day)
now_unix = int(time.mktime(date_time.timetuple())) * 1000
date_time = datetime.datetime(year, month, day + 10)
nex_unix = int(time.mktime(date_time.timetuple())) * 1000
print(now_unix, nex_unix)

time_delta = 15 * 60 * 1000
now_dir = 0

In [ ]:
beijing_nan = [116.3728, 39.8644]
beijing_box = [116.1530, 116.5732, 39.7737, 40.0455]

In [ ]:
def change_pemu(a, b):
    result = np.empty_like(a)
    result[b] = a
    return result

In [ ]:
def get_dis(a, b):
    return abs(a[0] - b[0] * 100 + abs(a[1] - b[1]) * 100

In [ ]:
def calc_time_delta(a, b):
    return (b - a) / time_delta

In [ ]:
def calculate_static(iteratrors):
    res = []
    res_id = []

    flag = 0
    for now__ in iteratrors:
        # print(now__)
        now_ = [now__[0], change_pemu(list(now__[1]), list(now__[6])),
                change_pemu(list(now__[2]), list(now__[6])),
                change_pemu(list(now__[3]), list(now__[6])),
                change_pemu(list(now__[4]), list(now__[6])),
                change_pemu(list(now__[5]), list(now__[6]))]

        cnt_in = 0
        cnt_ou = 0
        cnt_no = 96

        max_len = len(now_[1])

        for i in range(max_len):
            x = [now_[0], now_[1][i], now_[2][i], now_[3][i], now_[4][i], now_[5][i]]
            if get_dis([x[2], x[3]], beijing_nan) < 1:
                cnt_in += calc_time_delta(x[4], x[5])

            if x[2] > 0 and x[3] > 0 and (x[2] < beijing_box[0] or x[2] > beijing_box[1]
                    or x[3] < beijing_box[2] or x[3] > beijing_box[3]):
                cnt_ou += calc_time_delta(x[4], x[5])

            if x[2] > 0 and x[3] > 0:
                cnt_no -= calc_time_delta(x[4], x[5])

        if cnt_in > 1 and cnt_ou > 1 and cnt_no > 2:
            for i in range(max_len):
                x = [now_[0], now_[1][i], now_[2][i], now_[3][i], now_[4][i], now_[5][i]]
                res.append(x)
            res_id.append(now__[0])

    try:
        now_id = os.getpid()
        while True:
            path = ("/workdir/train_potential/%d/{}_data.npy" % now_dir).format(now_id)
            if os.path.exists(path):
                now_id += 20
            else:
                break

        np.save(("/workdir/train_potential/%d/{}_data.npy" % now_dir).format(now_id), res, allow_pickle=True)
        # np.save(("/workdir/train_potential/%d/{}_uid.npy" % now_dir).format(now_id), res_id, allow_pickle=True)
    except Exception as e:
        print(e)

In [ ]:
def output_data(full_data):
    target_columns = full_data.columns
    window_spec = Window.partitionBy("UID").orderBy("procedureStartTime")

    full_data = full_data.withColumn(
        "rownum",
        fn.row_number().over(window_spec) - 1
    )

    full_data = full_data.groupby("UID").agg(
        fn.collect_list("CID").alias("CID_list"),
        fn.collect_list("longitude").alias("longitude_list"),
        fn.collect_list("latitude").alias("latitude_list"),
        fn.collect_list("procedureStartTime").alias("procedureStartTime_list"),
        fn.collect_list("procedureEndTime").alias("procedureEndTime_list"),
        fn.collect_list("rownum").alias("rownum_list")
    )
    print("-----------------------------------------")
    full_data.foreachPartition(calculate_static)

In [ ]:
now_st = 0
now_len = 2000000
sub_data = 0
os.makedirs("/workdir/train_potential/%d" % sub_data, exist_ok=True)
now_dir = sub_data
sum_data = None
i = 1
data_dir = "/workdir/temp/202303%02d" % i
full_data = spark.read.parquet(data_dir)
sum_data = full_data.filter((full_data["UID"] < now_st + now_len) & (now_st <= full_data["UID"]))
output_data(sum_data)